In [8]:
import os
import cv2
import numpy as np
from ultralytics import YOLO
import json
import time
from sklearn.metrics import precision_score, recall_score, f1_score
import pandas as pd

# Load a pre-trained YOLOv8 model
best_model = YOLO('yolo11n.pt')

# Define traffic intensity thresholds
MILD_THRESHOLD = 4   # Number of vehicles for low traffic
STRONG_THRESHOLD = 6  # Number of vehicles for medium traffic

# Open the video
cap = cv2.VideoCapture('l1.mp4')

# Get frame dimensions
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

# Create the processed frames directory if it doesn't exist
output_folder = 'processed_frames'
os.makedirs(output_folder, exist_ok=True)

# Create the ground truth folder
ground_truth_folder = 'ground_truth'
os.makedirs(ground_truth_folder, exist_ok=True)

# Flags for mouse events
drawing = False
polygon_points = []

# Function to save AOI coordinates to a JSON file
def save_aoi_coordinates(points):
    with open('aoi_coordinates.json', 'w') as f:
        json.dump(points, f)
    print("AOI coordinates saved:", points)

# Mouse callback function to draw polygon
def draw_polygon(event, x, y, flags, param):
    global drawing, polygon_points
    
    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        polygon_points.append((x, y))
        save_aoi_coordinates(polygon_points)

    elif event == cv2.EVENT_MOUSEMOVE and drawing:
        frame_copy = frame.copy()
        if len(polygon_points) > 0:
            cv2.polylines(frame_copy, [np.array(polygon_points)], isClosed=False, color=(0, 255, 0), thickness=2)
        cv2.circle(frame_copy, (x, y), 5, (0, 0, 255), -1)
        cv2.imshow('Real-time Traffic Analysis', frame_copy)

    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False

# Create a named window and set mouse callback
cv2.namedWindow('Real-time Traffic Analysis')
cv2.setMouseCallback('Real-time Traffic Analysis', draw_polygon)

# Font and display settings
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 0.5
font_color = (255, 255, 255)

# Start time for FPS calculation
start_time = time.time()
frame_count = 0

# Initialize lists for metrics
all_ground_truths = []
all_predictions = []

# Data for Excel logging
excel_data = []

# Read video frames
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    detection_frame = frame.copy()
    results = best_model.predict(detection_frame, imgsz=640, conf=0.4)
    processed_frame = results[0].plot(line_width=1)

    if len(polygon_points) > 0:
        cv2.polylines(processed_frame, [np.array(polygon_points)], isClosed=True, color=(0, 255, 0), thickness=2)

    bounding_boxes = results[0].boxes
    vehicles_in_aoi = 0
    ground_truth = []
    predictions = []

    for box, class_id in zip(bounding_boxes.xyxy, bounding_boxes.cls):
        class_name = best_model.names[int(class_id)]
        box_center = ((box[0] + box[2]) / 2, (box[1] + box[3]) / 2)
        box_center = (int(box_center[0]), int(box_center[1]))
        
        ground_truth.append({'class': class_name, 'coordinates': [float(box[0]), float(box[1]), float(box[2]), float(box[3])]})
        predictions.append(1)

        if len(polygon_points) > 0 and cv2.pointPolygonTest(np.array(polygon_points, dtype=np.int32), box_center, False) >= 0:
            vehicles_in_aoi += 1

    ground_truth_path = os.path.join(ground_truth_folder, f'frame_{frame_count}.json')
    with open(ground_truth_path, 'w') as f:
        json.dump(ground_truth, f)

    if vehicles_in_aoi == 0:
        traffic_intensity = "No Traffic"
        font_color = (255, 0, 0)
    elif 1 <= vehicles_in_aoi <= MILD_THRESHOLD:
        traffic_intensity = "Low Traffic"
        font_color = (0, 165, 255)
    elif MILD_THRESHOLD < vehicles_in_aoi <= STRONG_THRESHOLD:
        traffic_intensity = "Medium Traffic"
        font_color = (0, 255, 255)
    else:
        traffic_intensity = "High Traffic"
        font_color = (0, 0, 255)

    cv2.putText(processed_frame, f'Vehicles in AOI: {vehicles_in_aoi}', (10, 50), font, font_scale, font_color, 2, cv2.LINE_AA)
    cv2.putText(processed_frame, f'Traffic Intensity: {traffic_intensity}', (10, 100), font, font_scale, font_color, 2, cv2.LINE_AA)

    output_frame_path = os.path.join(output_folder, f'processed_frame_{frame_count}.jpg')
    cv2.imwrite(output_frame_path, processed_frame)

    all_ground_truths.extend(ground_truth)
    all_predictions.extend(predictions)

    excel_data.append({'Frame': frame_count, 'Vehicles in AOI': vehicles_in_aoi, 'Traffic Intensity': traffic_intensity})

    frame_count += 1
    cv2.imshow('Real-time Traffic Analysis', processed_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

end_time = time.time()
elapsed_time = end_time - start_time
fps = frame_count / elapsed_time
print(f"Processed {frame_count} frames in {elapsed_time:.2f} seconds ({fps:.2f} FPS)")

# Metrics evaluation
ground_truth_labels = [1 if gt['class'] != 'unknown' else 0 for gt in all_ground_truths]
precision = precision_score(ground_truth_labels, all_predictions)
recall = recall_score(ground_truth_labels, all_predictions)
f1 = f1_score(ground_truth_labels, all_predictions)

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")

# Save data to Excel
df = pd.DataFrame(excel_data)
df.to_excel('traffic_analysis_log.xlsx', index=False)



0: 384x640 1 car, 202.2ms
Speed: 7.6ms preprocess, 202.2ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 207.8ms
Speed: 3.1ms preprocess, 207.8ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 196.9ms
Speed: 6.1ms preprocess, 196.9ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 192.6ms
Speed: 6.1ms preprocess, 192.6ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 198.9ms
Speed: 2.0ms preprocess, 198.9ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 204.7ms
Speed: 3.0ms preprocess, 204.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 199.9ms
Speed: 3.0ms preprocess, 199.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 193.2ms
Speed: 1.6ms preprocess, 193.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 6

In [10]:
import os
import cv2
import numpy as np
from ultralytics import YOLO
import json
import time
from sklearn.metrics import precision_score, recall_score, f1_score
import pandas as pd

# Load a pre-trained YOLOv8 model
best_model = YOLO('yolov8n.pt')

# Define traffic intensity thresholds
MILD_THRESHOLD = 4   # Number of vehicles for low traffic
STRONG_THRESHOLD = 6  # Number of vehicles for medium traffic

# Open the video
cap = cv2.VideoCapture('l1.mp4')

# Get frame dimensions
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

# Create the processed frames directory if it doesn't exist
output_folder = 'processed_frames'
os.makedirs(output_folder, exist_ok=True)

# Create the ground truth folder
ground_truth_folder = 'ground_truth'
os.makedirs(ground_truth_folder, exist_ok=True)

# Flags for mouse events
drawing = False
polygon_points = []

# Function to save AOI coordinates to a JSON file
def save_aoi_coordinates(points):
    with open('aoi_coordinates.json', 'w') as f:
        json.dump(points, f)
    print("AOI coordinates saved:", points)

# Mouse callback function to draw polygon
def draw_polygon(event, x, y, flags, param):
    global drawing, polygon_points
    
    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        polygon_points.append((x, y))
        save_aoi_coordinates(polygon_points)

    elif event == cv2.EVENT_MOUSEMOVE and drawing:
        frame_copy = frame.copy()
        if len(polygon_points) > 0:
            cv2.polylines(frame_copy, [np.array(polygon_points)], isClosed=False, color=(0, 255, 0), thickness=2)
        cv2.circle(frame_copy, (x, y), 5, (0, 0, 255), -1)
        cv2.imshow('Real-time Traffic Analysis', frame_copy)

    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False

# Create a named window and set mouse callback
cv2.namedWindow('Real-time Traffic Analysis')
cv2.setMouseCallback('Real-time Traffic Analysis', draw_polygon)

# Font and display settings
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 0.5
font_color = (255, 255, 255)

# Start time for FPS calculation
start_time = time.time()
frame_count = 0

# Initialize lists for metrics
all_ground_truths = []
all_predictions = []

# Data for Excel logging
excel_data = []

# Read video frames
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    detection_frame = frame.copy()
    results = best_model.predict(detection_frame, imgsz=640, conf=0.4)
    processed_frame = results[0].plot(line_width=1)

    if len(polygon_points) > 0:
        cv2.polylines(processed_frame, [np.array(polygon_points)], isClosed=True, color=(0, 255, 0), thickness=2)

    bounding_boxes = results[0].boxes
    vehicles_in_aoi = 0
    ground_truth = []
    predictions = []

    for box, class_id in zip(bounding_boxes.xyxy, bounding_boxes.cls):
        class_name = best_model.names[int(class_id)]
        box_center = ((box[0] + box[2]) / 2, (box[1] + box[3]) / 2)
        box_center = (int(box_center[0]), int(box_center[1]))
        
        ground_truth.append({'class': class_name, 'coordinates': [float(box[0]), float(box[1]), float(box[2]), float(box[3])]})
        predictions.append(1)

        if len(polygon_points) > 0 and cv2.pointPolygonTest(np.array(polygon_points, dtype=np.int32), box_center, False) >= 0:
            vehicles_in_aoi += 1

    ground_truth_path = os.path.join(ground_truth_folder, f'frame_{frame_count}.json')
    with open(ground_truth_path, 'w') as f:
        json.dump(ground_truth, f)

    if vehicles_in_aoi == 0:
        traffic_intensity = "No Traffic"
        font_color = (255, 0, 0)
    elif 1 <= vehicles_in_aoi <= MILD_THRESHOLD:
        traffic_intensity = "Low Traffic"
        font_color = (0, 165, 255)
    elif MILD_THRESHOLD < vehicles_in_aoi <= STRONG_THRESHOLD:
        traffic_intensity = "Medium Traffic"
        font_color = (0, 255, 255)
    else:
        traffic_intensity = "High Traffic"
        font_color = (0, 0, 255)

    cv2.putText(processed_frame, f'Vehicles in AOI: {vehicles_in_aoi}', (10, 50), font, font_scale, font_color, 2, cv2.LINE_AA)
    cv2.putText(processed_frame, f'Traffic Intensity: {traffic_intensity}', (10, 100), font, font_scale, font_color, 2, cv2.LINE_AA)

    output_frame_path = os.path.join(output_folder, f'processed_frame_{frame_count}.jpg')
    cv2.imwrite(output_frame_path, processed_frame)

    all_ground_truths.extend(ground_truth)
    all_predictions.extend(predictions)

    excel_data.append({'Frame': frame_count, 'Vehicles in AOI': vehicles_in_aoi, 'Traffic Intensity': traffic_intensity})

    frame_count += 1
    cv2.imshow('Real-time Traffic Analysis', processed_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

end_time = time.time()
elapsed_time = end_time - start_time
fps = frame_count / elapsed_time
print(f"Processed {frame_count} frames in {elapsed_time:.2f} seconds ({fps:.2f} FPS)")

# Metrics evaluation
ground_truth_labels = [1 if gt['class'] != 'unknown' else 0 for gt in all_ground_truths]
precision = precision_score(ground_truth_labels, all_predictions)
recall = recall_score(ground_truth_labels, all_predictions)
f1 = f1_score(ground_truth_labels, all_predictions)

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")

# Save data to Excel
df = pd.DataFrame(excel_data)
df.to_excel('traffic_analysis_log.xlsx', index=False)



0: 384x640 1 car, 289.8ms
Speed: 4.0ms preprocess, 289.8ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 227.0ms
Speed: 6.0ms preprocess, 227.0ms inference, 2.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 226.9ms
Speed: 4.0ms preprocess, 226.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 232.0ms
Speed: 3.0ms preprocess, 232.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 224.9ms
Speed: 3.0ms preprocess, 224.9ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 221.9ms
Speed: 4.5ms preprocess, 221.9ms inference, 5.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 219.8ms
Speed: 3.0ms preprocess, 219.8ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
AOI coordinates saved: [(158, 536)]

0: 384x640 2 cars, 226.7ms
Speed: 3.0ms preprocess, 226.7ms inference, 2.0ms postproc

In [11]:
import os
import cv2
import numpy as np
from ultralytics import YOLO
import json
import time
from sklearn.metrics import precision_score, recall_score, f1_score
import pandas as pd

# Load a pre-trained YOLOv8 model
best_model = YOLO('yolov5n.pt')

# Define traffic intensity thresholds
MILD_THRESHOLD = 4   # Number of vehicles for low traffic
STRONG_THRESHOLD = 6  # Number of vehicles for medium traffic

# Open the video
cap = cv2.VideoCapture('l1.mp4')

# Get frame dimensions
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

# Create the processed frames directory if it doesn't exist
output_folder = 'processed_frames'
os.makedirs(output_folder, exist_ok=True)

# Create the ground truth folder
ground_truth_folder = 'ground_truth'
os.makedirs(ground_truth_folder, exist_ok=True)

# Flags for mouse events
drawing = False
polygon_points = []

# Function to save AOI coordinates to a JSON file
def save_aoi_coordinates(points):
    with open('aoi_coordinates.json', 'w') as f:
        json.dump(points, f)
    print("AOI coordinates saved:", points)

# Mouse callback function to draw polygon
def draw_polygon(event, x, y, flags, param):
    global drawing, polygon_points
    
    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        polygon_points.append((x, y))
        save_aoi_coordinates(polygon_points)

    elif event == cv2.EVENT_MOUSEMOVE and drawing:
        frame_copy = frame.copy()
        if len(polygon_points) > 0:
            cv2.polylines(frame_copy, [np.array(polygon_points)], isClosed=False, color=(0, 255, 0), thickness=2)
        cv2.circle(frame_copy, (x, y), 5, (0, 0, 255), -1)
        cv2.imshow('Real-time Traffic Analysis', frame_copy)

    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False

# Create a named window and set mouse callback
cv2.namedWindow('Real-time Traffic Analysis')
cv2.setMouseCallback('Real-time Traffic Analysis', draw_polygon)

# Font and display settings
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 0.5
font_color = (255, 255, 255)

# Start time for FPS calculation
start_time = time.time()
frame_count = 0

# Initialize lists for metrics
all_ground_truths = []
all_predictions = []

# Data for Excel logging
excel_data = []

# Read video frames
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    detection_frame = frame.copy()
    results = best_model.predict(detection_frame, imgsz=640, conf=0.4)
    processed_frame = results[0].plot(line_width=1)

    if len(polygon_points) > 0:
        cv2.polylines(processed_frame, [np.array(polygon_points)], isClosed=True, color=(0, 255, 0), thickness=2)

    bounding_boxes = results[0].boxes
    vehicles_in_aoi = 0
    ground_truth = []
    predictions = []

    for box, class_id in zip(bounding_boxes.xyxy, bounding_boxes.cls):
        class_name = best_model.names[int(class_id)]
        box_center = ((box[0] + box[2]) / 2, (box[1] + box[3]) / 2)
        box_center = (int(box_center[0]), int(box_center[1]))
        
        ground_truth.append({'class': class_name, 'coordinates': [float(box[0]), float(box[1]), float(box[2]), float(box[3])]})
        predictions.append(1)

        if len(polygon_points) > 0 and cv2.pointPolygonTest(np.array(polygon_points, dtype=np.int32), box_center, False) >= 0:
            vehicles_in_aoi += 1

    ground_truth_path = os.path.join(ground_truth_folder, f'frame_{frame_count}.json')
    with open(ground_truth_path, 'w') as f:
        json.dump(ground_truth, f)

    if vehicles_in_aoi == 0:
        traffic_intensity = "No Traffic"
        font_color = (255, 0, 0)
    elif 1 <= vehicles_in_aoi <= MILD_THRESHOLD:
        traffic_intensity = "Low Traffic"
        font_color = (0, 165, 255)
    elif MILD_THRESHOLD < vehicles_in_aoi <= STRONG_THRESHOLD:
        traffic_intensity = "Medium Traffic"
        font_color = (0, 255, 255)
    else:
        traffic_intensity = "High Traffic"
        font_color = (0, 0, 255)

    cv2.putText(processed_frame, f'Vehicles in AOI: {vehicles_in_aoi}', (10, 50), font, font_scale, font_color, 2, cv2.LINE_AA)
    cv2.putText(processed_frame, f'Traffic Intensity: {traffic_intensity}', (10, 100), font, font_scale, font_color, 2, cv2.LINE_AA)

    output_frame_path = os.path.join(output_folder, f'processed_frame_{frame_count}.jpg')
    cv2.imwrite(output_frame_path, processed_frame)

    all_ground_truths.extend(ground_truth)
    all_predictions.extend(predictions)

    excel_data.append({'Frame': frame_count, 'Vehicles in AOI': vehicles_in_aoi, 'Traffic Intensity': traffic_intensity})

    frame_count += 1
    cv2.imshow('Real-time Traffic Analysis', processed_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

end_time = time.time()
elapsed_time = end_time - start_time
fps = frame_count / elapsed_time
print(f"Processed {frame_count} frames in {elapsed_time:.2f} seconds ({fps:.2f} FPS)")

# Metrics evaluation
ground_truth_labels = [1 if gt['class'] != 'unknown' else 0 for gt in all_ground_truths]
precision = precision_score(ground_truth_labels, all_predictions)
recall = recall_score(ground_truth_labels, all_predictions)
f1 = f1_score(ground_truth_labels, all_predictions)

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")

# Save data to Excel
df = pd.DataFrame(excel_data)
df.to_excel('traffic_analysis_log.xlsx', index=False)


PRO TIP  Replace 'model=yolov5n.pt' with new 'model=yolov5nu.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.


0: 384x640 1 car, 205.0ms
Speed: 6.2ms preprocess, 205.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 219.8ms
Speed: 4.0ms preprocess, 219.8ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 212.5ms
Speed: 5.7ms preprocess, 212.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 209.3ms
Speed: 3.3ms preprocess, 209.3ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 202.4ms
Speed: 3.0ms preprocess, 202.4ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 208.9ms
Speed: 5.0ms preprocess, 208.9ms inference, 2.0ms postprocess per image 